In [1]:
import numpy as np
import pandas as pd
import os
import glob
import superskrypt as sp
from scipy import stats

In [2]:
PTH = r"C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE"
os.chdir(PTH)
print(os.getcwd())

C:\Users\LEON-NETB\Dropbox\LEON_CIECHANOWSKI\DANE


In [8]:
df = "s1.txt"
df = pd.read_csv(df, sep='\t')
df = pd.DataFrame(df)

In [9]:
def is_odd(num):
    return num % 2

In [11]:
db

{'comp_l': [23, 35, 10, 35, 10, 35],
 'comp_r': [25, 4, 30, 6, 2, 6],
 'incomp_l': [20, 3, 2, 4, 30, 20],
 'incomp_r': [18, 50, 40, 15, 3, 4],
 'neut_l': [15, 6, 3, 30, 40, 10],
 'neut_r': [24, 2, 15, 10, 15, 30]}

In [12]:
db2

,comp_r_RT,comp_l_RT,incomp_r_RT,incomp_l_RT,neut_r_RT,neut_l_RT,comp_RT,incomp_RT,neut_RT
1,346.888889,338.439394,354.273973,363.176471,330.2,340.873016,342.664141,358.725222,335.536508


In [7]:
# counts SoA and RT for all subjects and concatenates them in a DataFrame 

db_ALL = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    print('read {}!'.format(f))
    db_SoA = sp.get_color_info(df)
    db_RT = sp.get_RT(df)
    db_ALL = db_ALL.append(db_SoA, ignore_index=True)
    for col in db_RT.columns:
        db_ALL.loc[0, '{}'.format(col)] = db_RT.loc[1, col]
print('Done!')

read s1.txt!
read s11.txt!
read s13.txt!


D:\Miniconda3\envs\py3\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


KeyboardInterrupt: 

In [27]:
# writing means of each condition (comp_l... etc):

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, c] = np.mean(db_ALL.loc[i, c])

In [28]:
# means of conditions of both hands:

for c in db_ALL.columns:
    for i in db_ALL.index:
        db_ALL.loc[i, 'comp'] = (db_ALL.loc[i, 'comp_l']+db_ALL.loc[i, 'comp_r'])/2
        db_ALL.loc[i, 'incomp'] = (db_ALL.loc[i, 'incomp_l']+db_ALL.loc[i, 'incomp_r'])/2
        db_ALL.loc[i, 'neut'] = (db_ALL.loc[i, 'neut_l']+db_ALL.loc[i, 'neut_r'])/2

In [29]:
db_ALL.index = subj_num
for i in db_ALL.index:
    if is_odd(i):
        db_ALL.loc[i, 'condition'] = 'info'
    else:
        db_ALL.loc[i, 'condition'] = 'noinfo'

ValueError: Length mismatch: Expected axis has 1 elements, new values have 0 elements

In [ ]:
db_ALL = db_ALL.loc[np.sort(db_ALL.index), :]
db_ALL.to_excel('BAZA_NONSTAND.xlsx')

In [ ]:
def get_zscores(df):
    inf = sp.get_color_info(df)
    newdf = pd.DataFrame(inf)
    all_ratings = newdf.values.ravel()
    df_zscore = newdf.copy()
    df_zscore.loc[:, :] = stats.zscore(all_ratings).reshape(newdf.shape)
    return df_zscore.mean()

In [ ]:
d1 = pd.DataFrame()
subj_num = []
for f in glob.glob('*.txt'):
    subj_num.append(int(f.split('.')[0][1:]))
    df = pd.read_csv(f, sep='\t')
    print('read {}!'.format(f))
    db = get_zscores(df)
    d1 = d1.append(db, ignore_index=True)

In [ ]:
for c in d1.columns:
    for i in d1.index:
        d1.loc[i, 'comp'] = (d1.loc[i, 'comp_l']+d1.loc[i, 'comp_r'])/2
        d1.loc[i, 'incomp'] = (d1.loc[i, 'incomp_l']+d1.loc[i, 'incomp_r'])/2
        d1.loc[i, 'neut'] = (d1.loc[i, 'neut_l']+d1.loc[i, 'neut_r'])/2

In [ ]:
d1.index = subj_num
for i in d1.index:
    if is_odd(i):
        d1.loc[i, 'condition'] = 'info'
    else:
        d1.loc[i, 'condition'] = 'noinfo'

In [ ]:
d1 = d1.loc[np.sort(d1.index), :]
d1.to_excel('BAZA_STANDARYZOWANA.xlsx')